In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import os
from os import path
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [6]:
# Chargement du jeu de données :
def load_signaux(dir):
    img_list = []
    class_counter = 0
    class_list = []
    for i in os.listdir(dir):
        if (path.isdir(path.join(dir, i))):
            for j in os.listdir(path.join(dir, i)):
                if (j.endswith('.ppm')):
                    class_list.append(class_counter)
                    img_list.append(np.array(Image.open(os.path.join(os.path.join(dir, i), j)).resize((28, 28))))
            class_counter += 1
    return np.array(img_list), np.array(class_list)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_dir = "train"
test_dir = "test"

In [9]:
X_train_full,y_train_full=load_signaux(train_dir)

In [10]:
X_test,y_test = load_signaux(test_dir)

In [11]:
X_train_full.shape

(4575, 28, 28, 3)

In [12]:
X_test.shape

(2520, 28, 28, 3)

In [13]:
y_test.shape

(2520,)

In [14]:
# Normalisation de l'input
X_train_full= X_train_full.astype('float32')
X_test_full= X_test.astype('float32')
X_train_full= X_train_full/ 255.0
X_test_full= X_test/ 255.0

In [15]:
# Categorical => like dummies
y_train_full= np_utils.to_categorical(y_train_full)
y_test_full= np_utils.to_categorical(y_test)


In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full,
                                                  y_train_full,
                                                  test_size=0.25, 
                                                  random_state=42, 
                                                  stratify=y_train_full)

In [18]:
num_classes= y_train_full.shape[1]

# Creation du modele

model = Sequential()
model.add(Conv2D(64, (4, 4), input_shape=(28,28,3), padding='same', activation='relu'))
model.add(Dropout(0.5))
model.add(keras.layers.AveragePooling2D())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
epochs= 20
lrate= 0.001
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        3136      
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 64)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               4

In [19]:
import datetime
Log_dir = "logs/fit/" + datetime.datetime.now().strftime("%m%d-%H%M%S")
Log_dir

'logs/fit/0119-164147'

In [20]:
#Log_dir=os.path.dirname(Log_dir)

callbacks = [
keras.callbacks.TensorBoard(Log_dir),
keras.callbacks.EarlyStopping(patience=5),
keras.callbacks.ModelCheckpoint("signaux_model_"+datetime.datetime.now().strftime("%m%d-%H%M%S")+".h5", save_best_only=True),
]

In [21]:
model.fit(
    X_train, y_train,
    epochs=40,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)

Epoch 1/40
108/108 [==============================] - 7s 60ms/step - loss: 3.6659 - accuracy: 0.1196 - val_loss: 2.3377 - val_accuracy: 0.4572
Epoch 2/40
108/108 [==============================] - 7s 63ms/step - loss: 2.1630 - accuracy: 0.4488 - val_loss: 1.1698 - val_accuracy: 0.6932
Epoch 3/40
108/108 [==============================] - 7s 62ms/step - loss: 1.3432 - accuracy: 0.6180 - val_loss: 0.7940 - val_accuracy: 0.7692
Epoch 4/40
108/108 [==============================] - 7s 61ms/step - loss: 0.9193 - accuracy: 0.7231 - val_loss: 0.5625 - val_accuracy: 0.8199
Epoch 5/40
108/108 [==============================] - 7s 67ms/step - loss: 0.7253 - accuracy: 0.7718 - val_loss: 0.4944 - val_accuracy: 0.8453
Epoch 6/40
108/108 [==============================] - 7s 68ms/step - loss: 0.6007 - accuracy: 0.8203 - val_loss: 0.3936 - val_accuracy: 0.8881
Epoch 7/40
108/108 [==============================] - 8s 70ms/step - loss: 0.4601 - accuracy: 0.8637 - val_loss: 0.3032 - val_accuracy: 0.9108

In [22]:
model.evaluate(X_test_full,y_test_full)

79/79 [==============================] - 1s 8ms/step - loss: 0.2427 - accuracy: 0.9421


[0.24266664683818817, 0.9420635104179382]

In [29]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets
